In [ ]:
!apt install pciutils

In [ ]:
!pip -q install langchain langchain-core langchain-community langchain-ollama beautifulsoup4 chromadb gradio pypdf

In [ ]:
!curl https://ollama.ai/install.sh | sh

In [1]:
import os  # Proporciona funciones para interactuar con el sistema operativo
import gradio as gr  # Gradio se utiliza para crear una interfaz web
import logging  # Para manejar los logs
import datetime  # Para registrar la fecha y hora de los logs
from langchain_community.vectorstores import Chroma  # Importar Chroma para cargar un almacén vectorial existente
from langchain_openai import OpenAIEmbeddings, ChatOpenAI  # Importar clases de OpenAI

# --- Configuración ---
# !!! IMPORTANTE: Cambia esto a la ruta real de tu base de datos Chroma !!!
CHROMA_DB_PATH = "/chroma_db" # Reemplaza con la ruta correcta
LOG_FILE = "app_log.txt"

# Configurar el sistema de logging
logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# --- Variables Globales ---
log_messages = []  # Almacena logs en memoria para mostrarlos en la UI
vectorstore = None # Almacenará la base de datos vectorial cargada
current_model = "openai"  # Modelo por defecto
finetuning_model_name = "" # Nombre del modelo de finetuning (si se usa)
openai_api_key_set = False # Flag para saber si la API key está configurada

# --- Funciones de Logging ---
def add_log(message, level="INFO"):
    """Añade un mensaje al log y lo guarda en memoria"""
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"{timestamp} - {level} - {message}"
    log_messages.append(log_entry)

    if level == "INFO":
        logging.info(message)
    elif level == "WARNING":
        logging.warning(message)
    elif level == "ERROR":
        logging.error(message)

    return log_entry

# --- Funciones Principales ---

def get_embeddings():
    """Obtiene el modelo de embeddings apropiado (solo OpenAI en esta versión)."""
    global current_model
    if not openai_api_key_set:
         # No lanzar error aquí, pero sí loguear. La carga del vectorstore fallará si no hay key.
         add_log("Advertencia: Intentando obtener embeddings sin configurar la API key de OpenAI.", "WARNING")
         # Devolver None o manejarlo en la función de carga
         return None

    add_log(f"Usando embeddings de OpenAI (modelo: text-embedding-3-small)")
    # Siempre usamos embeddings de OpenAI ya que Ollama fue eliminado
    # y Chroma DB requiere consistencia en los embeddings.
    return OpenAIEmbeddings(model="text-embedding-3-small")

def load_vectorstore():
    """Carga la base de datos vectorial Chroma desde la ruta especificada."""
    global vectorstore, CHROMA_DB_PATH
    add_log(f"Intentando cargar Chroma DB desde: {CHROMA_DB_PATH}")

    if not openai_api_key_set:
        error_msg = "Error: La API key de OpenAI no ha sido configurada en la pestaña 'Configuración de Modelo'."
        add_log(error_msg, "ERROR")
        return error_msg

    if not os.path.exists(CHROMA_DB_PATH):
        error_msg = f"Error: La ruta especificada para Chroma DB no existe: {CHROMA_DB_PATH}"
        add_log(error_msg, "ERROR")
        return error_msg

    embeddings = get_embeddings()
    if embeddings is None:
         error_msg = "Error: No se pudieron obtener los embeddings. Asegúrate de que la API key de OpenAI esté configurada."
         add_log(error_msg, "ERROR")
         return error_msg

    try:
        vectorstore = Chroma(persist_directory=CHROMA_DB_PATH, embedding_function=embeddings)
        success_msg = "Base de datos Chroma cargada correctamente."
        add_log(success_msg)
        return success_msg
    except Exception as e:
        error_msg = f"Error al cargar Chroma DB desde {CHROMA_DB_PATH}: {str(e)}"
        add_log(error_msg, "ERROR")
        vectorstore = None # Asegurarse de que esté None si falla la carga
        return error_msg

def call_llm(question, context):
    """Llama al modelo LLM seleccionado (OpenAI o Finetuning)."""
    global current_model, finetuning_model_name
    add_log(f"Enviando pregunta al modelo ({current_model}): {question[:50]}...")
    formatted_prompt = f"""Usa únicamente el siguiente contexto para responder la pregunta en español. Si el contexto no contiene la respuesta, indica que no puedes responder con la información proporcionada.
    Pregunta: {question}

    Contexto:
    {context}
    """
    add_log("Esperando respuesta del modelo")

    try:
        if current_model == "openai":
            llm = ChatOpenAI(model="gpt-4o-mini") # Modelo base de OpenAI
            response = llm.invoke(formatted_prompt)
            result = response.content
        elif current_model == "finetuning":
            if not finetuning_model_name:
                 add_log("Error: Modelo 'finetuning' seleccionado pero no se ha proporcionado un nombre de modelo.", "ERROR")
                 return "Error: Falta el nombre del modelo de finetuning."
            add_log(f"Usando modelo de finetuning: {finetuning_model_name}")
            llm = ChatOpenAI(model=finetuning_model_name) # Modelo de finetuning
            response = llm.invoke(formatted_prompt)
            result = response.content
        else:
            # Este caso no debería ocurrir ahora que Ollama está eliminado
            add_log(f"Error: Modelo desconocido seleccionado: {current_model}", "ERROR")
            return f"Error: Modelo no soportado: {current_model}"

        add_log("Respuesta recibida del modelo")
        return result
    except Exception as e:
        error_msg = f"Error al llamar al LLM ({current_model}): {str(e)}"
        add_log(error_msg, "ERROR")
        # Podrías querer verificar si el error es por la API key aquí también
        if "api key" in str(e).lower():
            return "Error: Problema con la API key de OpenAI. Verifícala en la configuración."
        return f"Error al contactar el modelo: {str(e)}"

def rag_chain(question):
    """Ejecuta la cadena RAG usando el vectorstore global cargado."""
    global vectorstore
    SEARCH_TYPE = "mmr"  # Tipo de búsqueda (Maximal Marginal Relevance)
    SEARCH_KWARGS = {"k": 5, "fetch_k": 50}  # Parámetros de búsqueda

    if vectorstore is None:
        add_log("Error: Vectorstore no cargado. Intenta inicializar el chat.", "ERROR")
        return "Error: La base de datos vectorial no está cargada. Por favor, inicializa el chat primero desde 'Opciones de inicialización'."

    add_log(f"Iniciando cadena RAG con la configuracion, search_type : {SEARCH_TYPE}, search_kwargs: {SEARCH_KWARGS}")
    try:
        retriever = vectorstore.as_retriever(
            search_type=SEARCH_TYPE,
            search_kwargs=SEARCH_KWARGS
        )
        add_log("Recuperando documentos relevantes")
        retrieved_docs = retriever.invoke(question)
        add_log(f"Recuperados {len(retrieved_docs)} documentos relevantes")

        if not retrieved_docs:
             add_log("No se encontraron documentos relevantes en la base de datos para la pregunta.", "WARNING")
             # Podrías devolver un mensaje específico o intentar llamar al LLM sin contexto
             # return "No encontré información relevante en la base de datos para responder a tu pregunta."
             formatted_context = "No se encontró información relevante en la base de datos."
        else:
            formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)

        return call_llm(question, formatted_context) # Obtener la respuesta del modelo

    except Exception as e:
        error_msg = f"Error durante la ejecución de RAG: {str(e)}"
        add_log(error_msg, "ERROR")
        return f"Error en RAG: {str(e)}"


# --- Funciones de la Interfaz (Gradio) ---

def chat_response(message, history):
    """Maneja la respuesta del chat."""
    global vectorstore

    # Verificar si el vectorstore está cargado
    if vectorstore is None:
        response = "La base de datos vectorial no está cargada. Por favor, ve a 'Opciones de inicialización' y haz clic en 'Inicializar Chat / Cargar Base de Datos' después de configurar el modelo."
        add_log("Intento de chat sin vectorstore cargado.", "WARNING")
    else:
        add_log(f"Nueva consulta de chat recibida: {message}")
        try:
            add_log("Obteniendo respuesta para el chat vía RAG")
            response = rag_chain(message) # Usar la cadena RAG
            add_log("Respuesta de chat generada correctamente")
        except Exception as e:
            error_msg = f"Error al procesar la consulta de chat: {str(e)}"
            add_log(error_msg, "ERROR")
            response = f"Error: {error_msg}"

    # Añadir al historial y devolver
    history.append((message, response))
    return "", history # Limpiar input y devolver historial actualizado

def change_model(model_choice, api_key=None, finetuning_name=None):
    """Cambia el modelo actual y configura la API key si es necesario."""
    global current_model, finetuning_model_name, openai_api_key_set, vectorstore

    # Resetear estado al cambiar de modelo
    openai_api_key_set = False
    vectorstore = None # Forzar recarga del vectorstore con los nuevos embeddings/configuración
    status_message = ""

    current_model = model_choice
    add_log(f"Cambiando modelo a: {model_choice}")

    if model_choice in ["openai", "finetuning"]:
        if api_key:
            os.environ["OPENAI_API_KEY"] = api_key
            openai_api_key_set = True
            add_log("API key de OpenAI configurada.")
            status_message += f"Modelo cambiado a: {model_choice}. API Key configurada. "
        else:
            add_log("Advertencia: Modelo OpenAI/Finetuning seleccionado pero no se proporcionó API key.", "WARNING")
            status_message += f"Modelo cambiado a: {model_choice}. ¡Advertencia! Falta API Key. "

    if model_choice == "finetuning":
        if finetuning_name:
            finetuning_model_name = finetuning_name
            add_log(f"Nombre del modelo de finetuning configurado: {finetuning_name}")
            status_message += f"Modelo Finetuning: {finetuning_name}. "
        else:
            add_log("Advertencia: Modelo 'finetuning' seleccionado pero no se proporcionó nombre.", "WARNING")
            status_message += "¡Advertencia! Falta el nombre del modelo Finetuning. "

    status_message += "Recuerda inicializar el chat para cargar la base de datos con la nueva configuración."
    add_log(status_message)
    return status_message

def update_fields_visibility(model_choice):
    """Actualiza la visibilidad de los campos de API Key y Finetuning."""
    if model_choice == "openai":
        # Mostrar API Key, ocultar Finetuning
        return gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)
    elif model_choice == "finetuning":
        # Mostrar API Key y Finetuning
        return gr.update(visible=True), gr.update(visible=True), gr.update(visible=True)
    else:
        # Ocultar ambos (aunque no debería haber otros modelos ahora)
         return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)


def get_logs():
    """Obtiene los logs actuales para mostrar en la UI."""
    return "\n".join(log_messages)

# --- Interfaz Gráfica (Gradio) ---
with gr.Blocks(theme=gr.themes.Soft()) as iface:
    gr.Markdown("# RAG con Base de Datos Chroma y Modelos OpenAI")
    gr.Markdown(f"Haz preguntas sobre el contenido de una base de datos Chroma predefinida ubicada en: `{CHROMA_DB_PATH}`. El sistema usará RAG para responder.")

    with gr.Tabs():
        with gr.TabItem("Chat Interactivo"):
            with gr.Row():
                with gr.Column():
                    # Área de chat
                    chatbot = gr.Chatbot(height=400, label="Conversación")

                    # Controles de inicialización
                    with gr.Accordion("Opciones de inicialización", open=True):
                        init_chat_db_btn = gr.Button("Inicializar Chat / Cargar Base de Datos", variant="primary")
                        chat_init_status = gr.Textbox(label="Estado de inicialización", lines=2, interactive=False)

                    # Área de entrada de mensaje
                    msg_input = gr.Textbox(placeholder="Escribe tu pregunta aquí...", label="Tu mensaje")
                    with gr.Row():
                        submit_btn = gr.Button("Enviar", variant="primary")
                        clear_btn = gr.Button("Limpiar chat")

            # Configurar eventos para el chat
            submit_btn.click(
                fn=chat_response,
                inputs=[msg_input, chatbot],
                outputs=[msg_input, chatbot]
            )

            clear_btn.click(lambda: (None, []), None, [msg_input, chatbot], queue=False)

            # Evento para inicializar el chat (cargar DB)
            init_chat_db_btn.click(
                fn=load_vectorstore,
                inputs=None, # No necesita inputs directos, usa globales
                outputs=[chat_init_status]
            )

        with gr.TabItem("Configuración de Modelo"):
            gr.Markdown("### Selecciona el modelo LLM a utilizar")
            model_choice = gr.Radio(
                choices=["openai", "finetuning"], # Ollama eliminado
                value="openai", # Default a OpenAI
                label="Modelo para LLM y Embeddings (Embeddings siempre OpenAI)"
            )

            # Campos para OpenAI y Finetuning
            api_key_input = gr.Textbox(
                placeholder="Ingresa tu API key de OpenAI aquí...",
                label="OpenAI API Key",
                type="password",
                visible=True # Visible por defecto ya que openai es el default
            )

            finetuning_model_input = gr.Textbox(
                placeholder="Nombre del modelo de finetuning (ej: ft:gpt-3.5-turbo:...)",
                label="Nombre del Modelo de Finetuning",
                visible=False # Oculto por defecto
            )

            finetuning_info = gr.Markdown(
                """
                ### Información sobre Finetuning
                Para usar un modelo de finetuning de OpenAI, necesitas:
                1. Tu API key de OpenAI (en el campo de arriba)
                2. El nombre completo del modelo de finetuning (formato: ft:...)
                """,
                visible=False # Oculto por defecto
            )

            model_status = gr.Textbox(label="Estado de la configuración", lines=2, interactive=False)

            # Botón para aplicar cambios
            apply_model_btn = gr.Button("Aplicar configuración", variant="primary")

            # Evento para cambiar la visibilidad de los campos
            model_choice.change(
                fn=update_fields_visibility,
                inputs=[model_choice],
                outputs=[api_key_input, finetuning_model_input, finetuning_info]
            )

            # Evento para aplicar la configuración del modelo
            apply_model_btn.click(
                fn=change_model, # Llama a la función refactorizada
                inputs=[model_choice, api_key_input, finetuning_model_input],
                outputs=[model_status]
            )

            gr.Markdown("""
            ### Información Importante
            - **Modelo**: Selecciona si usarás el modelo base de OpenAI (`gpt-4o-mini`) o un modelo personalizado (`finetuning`).
            - **Embeddings**: Siempre se usarán los embeddings de OpenAI (`text-embedding-3-small`) para cargar y consultar la base de datos Chroma, independientemente del modelo LLM seleccionado. Esto es necesario para la consistencia.
            - **API Key**: Necesitas una API key de OpenAI válida para ambos modelos.
            - **Inicialización**: Después de aplicar la configuración, ve a la pestaña "Chat Interactivo" y haz clic en "Inicializar Chat / Cargar Base de Datos" para cargar la base de datos con la configuración correcta.
            """)

        with gr.TabItem("Logs"):
            refresh_btn = gr.Button("Actualizar Logs")
            logs_output = gr.Textbox(label="📋 Logs del Sistema", lines=20, interactive=False)

            # Evento para logs
            refresh_btn.click(get_logs, inputs=None, outputs=logs_output)

# --- Inicio de la Aplicación ---
add_log("Aplicación iniciada")
# Lanzar la aplicación Gradio
if __name__ == "__main__":
    # Puedes añadir argumentos de línea de comando aquí si lo necesitas
    # Por ejemplo, para especificar la ruta de la DB:
    # import argparse
    # parser = argparse.ArgumentParser()
    # parser.add_argument("--chroma_db_path", default="path/to/your/chroma_db", help="Ruta a la base de datos Chroma")
    # args = parser.parse_args()
    # CHROMA_DB_PATH = args.chroma_db_path

    iface.launch(share=True) # share=True para acceso público temporal si es necesario


ModuleNotFoundError: No module named 'gradio'

In [ ]:
import os
import asyncio

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

In [ ]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

In [ ]:
import re
import os # Importado para manejar nombres de archivo y rutas
from chromadb import PersistentClient
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# --- Constantes y Configuración ---

# Estructura para definir los PDFs a procesar y sus filtros de fecha específicos.
# Cada diccionario debe tener:
#   - "ruta": La ruta al archivo PDF.
#   - "fechas": Una lista de cadenas de texto con las fechas a buscar en ESE archivo,
#               o None/lista vacía si NO se debe filtrar por fecha para ese archivo.
PDFS_A_PROCESAR = [
    {
        "ruta": "Ley Actualizada 2025 .pdf",
        "fechas": None # Buscar estas fechas en este archivo
    },
    {
        "ruta": "Ley del 2024 .pdf",
        "fechas": None # No filtrar por fecha en este archivo
    },
    # Agrega más diccionarios aquí para otros PDFs
]

CHUNK_SIZE = 1000 # Tamaño de los fragmentos de texto
CHUNK_OVERLAP = 100 # Superposición entre fragmentos consecutivos
VECTORSTORE_DIR = "./chroma_db" # Directorio para guardar la base de datos vectorial
CURRENT_MODEL = "openai" # Modelo de embeddings a usar
COLLECTION_NAME = "leyes_mexico_selectivo" # Nombre para la colección en ChromaDB


# --- Funciones de Procesamiento de Texto y PDF ---

def limpiar_texto(texto):
    """Elimina espacios en blanco excesivos y líneas vacías."""
    texto = re.sub(r'\s+', ' ', texto) # Reemplaza múltiples espacios/saltos con uno solo
    texto = re.sub(r'\n\s*\n', '\n\n', texto) # Normaliza saltos de línea dobles
    texto = re.sub(r'(Artículo\s+\d+)o(?=[.-])', r'\1', texto) # Añadido IGNORECASE por si acaso
    return texto.strip() # Elimina espacios al inicio y final

def dividir_texto(texto, chunk_size=1000, chunk_overlap=100):
    """Divide el texto en fragmentos superpuestos."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(texto)

def extraer_texto_sin_encabezado_pie(ruta_pdf, margen_superior=10, margen_inferior=10):
    """
    Extrae texto del PDF excluyendo un porcentaje superior e inferior de cada página
    para evitar encabezados y pies de página.
    """
    texto_completo = ""
    try:
        documento = fitz.open(ruta_pdf)
        print(f"Procesando archivo: {ruta_pdf}")
        for i, pagina in enumerate(documento):
            altura = pagina.rect.height
            anchura = pagina.rect.width
            # Define el rectángulo que representa el área de contenido principal
            rect_contenido = fitz.Rect(
                0,
                altura * margen_superior / 100.0, # Coordenada Y superior
                anchura,
                altura * (1 - margen_inferior / 100.0) # Coordenada Y inferior
            )
            # Extrae el texto solo dentro del rectángulo de contenido
            texto_pagina = pagina.get_text("text", clip=rect_contenido)
            texto_limpio_pagina = limpiar_texto(texto_pagina)
            if texto_limpio_pagina: # Solo añade si hay texto después de limpiar
                 texto_completo += texto_limpio_pagina + "\n\n" # Añade separación entre páginas

        documento.close()
    except Exception as e:
        print(f"Error al procesar el archivo {ruta_pdf}: {e}")
        return "" # Devuelve cadena vacía si hay error
    return texto_completo

def extraer_articulos_para_rag(texto_completo, nombre_archivo, fechas=None, chunk_size=1000, chunk_overlap=100):
    """
    Extrae artículos de la ley, filtra por fecha SI SE ESPECIFICA ('fechas' no es None ni vacío),
    los divide en fragmentos y añade metadatos, incluyendo el nombre del archivo fuente.
    """
    # Expresión regular para encontrar bloques que empiezan con "Artículo \d+.*.-"
    # y terminan justo antes del siguiente "Artículo \d+.*.-" o al final del texto.
    # articulos = re.findall(r"(Artículo \d+.*?.-)(.*?)(?=Artículo \d+.*?.-|$)", texto_completo, re.DOTALL)
    articulos = re.findall(r"(Artículo \d+.*?)\s*(-|\.)\s*(.*?)(?=Artículo \d+.*?[-.]|$)", texto_completo, re.DOTALL)
    resultados = []
    for articulo in articulos:
        encabezado = articulo[0].strip() + articulo[1].strip()
        contenido = articulo[2].strip()
        resultados.append((encabezado, contenido))
    fragmentos_con_metadatos = []

    # Asegurarse de que 'fechas' sea una lista o None. Si es lista vacía, se tratará como None.
    if isinstance(fechas, list) and not fechas:
        fechas = None

    # Imprimir si se aplicará filtro de fecha o no
    if fechas:
        print(f"  Buscando artículos en '{nombre_archivo}' que contengan las fechas: {', '.join(fechas)}")
    else:
        print(f"  Extrayendo todos los artículos de '{nombre_archivo}' (sin filtro de fecha).")

    
    articulos_encontrados_en_archivo = 0
    for encabezado, contenido in resultados:
        procesar_articulo = True # Por defecto, procesar el artículo

        # Aplicar filtro de fecha solo si 'fechas' es una lista con elementos
        if fechas:
            fecha_encontrada = False
            for fecha in fechas:
                # Búsqueda insensible a mayúsculas/minúsculas en el contenido
                if re.search(re.escape(fecha), contenido, re.IGNORECASE):
                    fecha_encontrada = True
                    break # Si encuentra una fecha, no necesita buscar las otras en este artículo

            if not fecha_encontrada:
                procesar_articulo = False # No procesar si no se encontró ninguna fecha requerida

        # Si pasa el filtro de fecha (o no había filtro), procesar el artículo
        if procesar_articulo:
            # Extraer el número del artículo del encabezado
            match_numero = re.search(r'\d+', encabezado)
            numero_articulo = int(match_numero.group(0)) if match_numero else 0

            articulos_encontrados_en_archivo += 1
            nombre_archivo_sin_extension = os.path.splitext(nombre_archivo)[0]

            texto_articulo = nombre_archivo_sin_extension + encabezado + contenido # Combina encabezado y contenido
            chunks = dividir_texto(texto_articulo, chunk_size, chunk_overlap)

            for i, chunk in enumerate(chunks):
                # Si es un chunk posterior al primero y no empieza ya con el encabezado, añadirlo
                # para dar contexto.
                if i > 0 and not chunk.strip().lower().startswith(encabezado.strip().lower()):
                     # Añade el encabezado y una marca de continuación
                    chunk_con_contexto = f"{encabezado.strip()} (continuación)... {chunk.strip()}"
                else:
                    chunk_con_contexto = chunk # Usa el chunk tal cual si es el primero o ya tiene el encabezado

                # Crear metadatos para este chunk
                metadatos = {
                    "titulo": encabezado.strip(), # Título del artículo
                    "numero_articulo": numero_articulo, # Título del artículo
                    "fuente": nombre_archivo_sin_extension, # Nombre del archivo PDF de origen
                    "parte": i + 1, # Número de parte del artículo
                    "total_partes": len(chunks) # Total de partes en que se dividió el artículo
                }
                # Añadir el fragmento y sus metadatos a la lista
                fragmentos_con_metadatos.append({"texto": chunk_con_contexto, "metadatos": metadatos})

    print(f"  Se procesaron {articulos_encontrados_en_archivo} artículos relevantes en '{nombre_archivo_sin_extension}'.")
    return fragmentos_con_metadatos

# --- Funciones de LangChain y ChromaDB ---

def crear_documentos(fragmentos_con_metadatos):
    """Convierte los fragmentos (diccionarios) en objetos Document de LangChain."""
    return [Document(page_content=frag["texto"], metadata=frag["metadatos"])
            for frag in fragmentos_con_metadatos]

def get_embeddings():
    """Obtiene el modelo de embeddings configurado."""
    global CURRENT_MODEL
    if CURRENT_MODEL == "openai":
        if not OPENAI_API_KEY: # Verifica si la clave existe (aunque esté codificada)
             raise ValueError("La clave API de OpenAI no está definida.")
        # Inicializa OpenAIEmbeddings con el modelo especificado y la clave API
        return OpenAIEmbeddings(model="text-embedding-3-small", api_key=OPENAI_API_KEY)
    # Aquí podrías añadir lógica para otros proveedores de embeddings si fuera necesario
    # elif CURRENT_MODEL == "huggingface":
    #     from langchain_community.embeddings import HuggingFaceEmbeddings
    #     return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    else:
        raise ValueError(f"Modelo de embeddings no soportado: {CURRENT_MODEL}")

def crear_vectorstore(documentos, persist_directory, collection_name):
    """Crea y guarda un vectorstore Chroma con los documentos procesados."""
    if not documentos:
        print("No hay documentos para añadir al vectorstore.")
        return None

    try:
        embeddings = get_embeddings() # Obtiene el modelo de embeddings
        print(f"Creando/actualizando vectorstore en '{persist_directory}' con la colección '{collection_name}'...")
        # Crea el vectorstore a partir de los documentos
        vectorstore = Chroma.from_documents(
            documents=documentos,
            embedding=embeddings,
            persist_directory=persist_directory, # Directorio donde se guardará
            collection_name=collection_name # Nombre de la colección dentro de ChromaDB
        )
        print("Vectorstore creado/actualizado exitosamente.")
        return vectorstore
    except Exception as e:
        print(f"Error al crear el vectorstore: {e}")
        # Podrías querer manejar diferentes tipos de errores aquí
        # (ej. error de API key, error de escritura en disco)
        return None

def delete_collection(collection_name, persist_directory):
    """Elimina una colección específica de la base de datos Chroma."""
    try:
        print(f"Intentando eliminar la colección '{collection_name}' de '{persist_directory}'...")
        # Crea un cliente Chroma apuntando al directorio persistente
        chroma_client = PersistentClient(path=persist_directory)
        # Intenta eliminar la colección
        chroma_client.delete_collection(collection_name)
        print(f"Colección '{collection_name}' eliminada exitosamente.")
    except Exception as e:
        # Captura cualquier excepción, incluyendo si la colección no existe
        print(f"No se pudo eliminar la colección '{collection_name}': {e}")
        # Podrías querer ser más específico si sabes qué excepciones esperar
        # (ej. ValueError si la colección no existe en algunas versiones de Chroma)


# --- Función Principal ---

def main():
    """Flujo principal del script: procesa PDFs y crea el vectorstore."""

    # Opcional: Eliminar la colección existente antes de empezar
    delete_collection(COLLECTION_NAME, VECTORSTORE_DIR) # Descomenta si quieres empezar de cero cada vez

    todos_los_fragmentos_rag = [] # Lista para acumular fragmentos de todos los PDFs

    # Itera sobre cada diccionario de configuración de PDF
    for config_pdf in PDFS_A_PROCESAR:
        ruta_pdf = config_pdf.get("ruta")
        fechas_filtro = config_pdf.get("fechas") # Puede ser None, lista vacía o lista con fechas

        if not ruta_pdf:
            print("Advertencia: Se encontró una configuración sin 'ruta'. Saltando...")
            continue

        if not os.path.exists(ruta_pdf):
             print(f"Advertencia: El archivo '{ruta_pdf}' no se encontró. Saltando...")
             continue # Pasa al siguiente archivo si este no existe

        # 1. Extraer texto limpio del PDF actual
        texto_completo_pdf = extraer_texto_sin_encabezado_pie(ruta_pdf)
        if not texto_completo_pdf:
            print(f"No se pudo extraer texto de '{ruta_pdf}' o está vacío. Saltando...")
            continue # Pasa al siguiente archivo si no se extrajo texto

        # 2. Extraer fragmentos, aplicando el filtro de fecha específico para este archivo
        fragmentos_pdf = extraer_articulos_para_rag(
            texto_completo_pdf,
            nombre_archivo=os.path.basename(ruta_pdf), # Pasa solo el nombre del archivo como fuente
            fechas=fechas_filtro, # Pasa las fechas específicas (o None)
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP
        )

        # 3. Añadir los fragmentos de este PDF a la lista general
        todos_los_fragmentos_rag.extend(fragmentos_pdf)
        print(f"Se añadieron {len(fragmentos_pdf)} fragmentos del archivo '{ruta_pdf}'.\n")

    # Verifica si se recopilaron fragmentos en total
    if not todos_los_fragmentos_rag:
        print("No se encontraron fragmentos relevantes en ninguno de los archivos PDF procesados.")
        return # Termina si no hay nada que añadir al vectorstore

    print(f"\nTotal de fragmentos recopilados de todos los archivos: {len(todos_los_fragmentos_rag)}")

    # 4. Convertir todos los fragmentos recopilados a documentos LangChain
    documentos_langchain = crear_documentos(todos_los_fragmentos_rag)
    print(f"Se crearon {len(documentos_langchain)} documentos LangChain.")

    # 5. Crear y guardar el vectorstore con todos los documentos
    vectorstore = crear_vectorstore(documentos_langchain, VECTORSTORE_DIR, collection_name=COLLECTION_NAME)

    if vectorstore:
        print(f"\nProceso completado. Vectorstore ('{COLLECTION_NAME}') guardado/actualizado en '{VECTORSTORE_DIR}'.")
    else:
        print("\nEl proceso finalizó, pero hubo un error al crear el vectorstore.")

# --- Punto de Entrada del Script ---
if __name__ == "__main__":
    main()


Intentando eliminar la colección 'leyes_mexico_selectivo' de './chroma_db'...
Colección 'leyes_mexico_selectivo' eliminada exitosamente.
Procesando archivo: Ley Actualizada 2025 .pdf
  Extrayendo todos los artículos de 'Ley Actualizada 2025 .pdf' (sin filtro de fecha).
  Se procesaron 40 artículos relevantes en 'Ley Actualizada 2025 '.
Se añadieron 90 fragmentos del archivo 'Ley Actualizada 2025 .pdf'.

Procesando archivo: Ley del 2024 .pdf
  Extrayendo todos los artículos de 'Ley del 2024 .pdf' (sin filtro de fecha).
  Se procesaron 1318 artículos relevantes en 'Ley del 2024 '.
Se añadieron 2284 fragmentos del archivo 'Ley del 2024 .pdf'.


Total de fragmentos recopilados de todos los archivos: 2374
Se crearon 2374 documentos LangChain.
Creando/actualizando vectorstore en './chroma_db' con la colección 'leyes_mexico_selectivo'...
Vectorstore creado/actualizado exitosamente.

Proceso completado. Vectorstore ('leyes_mexico_selectivo') guardado/actualizado en './chroma_db'.


In [29]:
import chromadb
import pprint # Para imprimir diccionarios de forma más legible

# --- 1. Configuración del Cliente ---
# Opcion A: Cliente en memoria (los datos se pierden al cerrar)
# cliente = chromadb.Client()

VECTORSTORE_DIR = "./chroma_db" # Directorio para guardar la base de datos vectorial
COLLECTION_NAME = "leyes_mexico_selectivo" # Nombre para la colección en ChromaDB
# Opcion B: Cliente persistente (guarda los datos en disco)
# Asegúrate de que el directorio 'mi_base_chroma' exista o pueda ser creado.
ruta_persistencia = VECTORSTORE_DIR
cliente = chromadb.PersistentClient(path=ruta_persistencia)

# --- 2. Nombre de tu colección ---
nombre_coleccion = COLLECTION_NAME # ¡¡IMPORTANTE: Reemplaza esto con el nombre real de tu colección!!

try:
    # --- 3. Obtener la Colección ---
    print(f"Intentando acceder a la colección: '{nombre_coleccion}'...")
    coleccion = cliente.get_collection(name=nombre_coleccion)
    print(f"Acceso exitoso a la colección '{nombre_coleccion}'.")

    # --- Opcional: Ver cuántos ítems hay ---
    count = coleccion.count()
    print(f"La colección tiene {count} ítems.")

    # --- 4. Recuperar datos de la colección ---
    # Puedes ajustar los parámetros de get() según necesites:
    # - ids: Una lista de IDs específicos a recuperar.
    # - where: Un filtro de metadatos (ej: {"metadata_field": "value"}).
    # - limit: Número máximo de resultados a devolver.
    # - offset: Empezar a devolver resultados desde esta posición.
    # - include: Lista especificando qué retornar: ["metadatas", "documents", "embeddings"]

    print("\nRecuperando datos (primeros 100, incluyendo documentos y metadatos)...")
    resultados = coleccion.get(
        limit=3000,           # Limita a los primeros 10 para no imprimir demasiado
        include=["metadatas", "documents"] # Especifica qué quieres ver
        # Si también quieres los embeddings (vectores), añade "embeddings":
        # include=["metadatas", "documents", "embeddings"]
        # Ten cuidado, los embeddings pueden ser muy largos para imprimir.
    )

    # --- 5. Imprimir los Resultados ---
    print("\n--- Contenido Recuperado ---")
    if resultados and resultados.get('ids'): # Verifica si se devolvieron resultados
        # Imprime de forma más estructurada
        num_resultados = len(resultados['ids'])
        print(f"Mostrando {num_resultados} de {count} ítems:")

        for i in range(num_resultados):
            print(f"\nÍtem {i+1}:")
            print(f"  ID: {resultados['ids'][i]}")

            if resultados.get('documents') and resultados['documents'][i] is not None:
                 # Usa repr() para mostrar saltos de línea y caracteres especiales
                print(f"  Documento: {repr(resultados['documents'][i])}")
            else:
                print("  Documento: (No solicitado o no disponible)")

            if resultados.get('metadatas') and resultados['metadatas'][i] is not None:
                print( "  Metadatos:")
                pprint.pprint(resultados['metadatas'][i], indent=4) # Imprime el diccionario de metadatos
            else:
                print("  Metadatos: (No solicitados o no disponibles)")

            # Descomenta si incluiste embeddings y quieres imprimirlos (serán largos)
            # if resultados.get('embeddings') and resultados['embeddings'][i] is not None:
            #     print(f"  Embedding: {resultados['embeddings'][i][:5]}... (primeros 5 valores)") # Imprime solo una parte
            # else:
            #     print("  Embedding: (No solicitado o no disponible)")

    else:
        print("No se encontraron datos en la colección con los criterios dados o la colección está vacía.")

except Exception as e:
    print(f"\nError: {e}")
    print("Verifica que el nombre de la colección sea correcto y que el cliente ChromaDB esté configurado adecuadamente.")
    # Opcional: Listar colecciones disponibles si falla el get_collection
    try:
        lista_colecciones = cliente.list_collections()
        print("\nColecciones disponibles:")
        if lista_colecciones:
            for col in lista_colecciones:
                print(f"- {col.name}")
        else:
            print("(Ninguna)")
    except Exception as list_e:
        print(f"No se pudieron listar las colecciones: {list_e}")

Intentando acceder a la colección: 'leyes_mexico_selectivo'...
Acceso exitoso a la colección 'leyes_mexico_selectivo'.
La colección tiene 2367 ítems.

Recuperando datos (primeros 100, incluyendo documentos y metadatos)...

--- Contenido Recuperado ---
Mostrando 2367 de 2367 ítems:

Ítem 1:
  ID: 81abae26-5daf-4231-9391-7d3969fdedd4
  Documento: 'Artículo 1.- La presente Ley es de observancia general en toda la República y rige las relaciones de trabajo comprendidas en el artículo 123, Apartado A, de la Constitución.'
  Metadatos:
{'numero_articulo': 1, 'parte': 1, 'titulo': 'Artículo 1.', 'total_partes': 1}

Ítem 2:
  ID: 6a661ebb-6080-4b8c-9592-31dce5986ac3
  Documento: 'Artículo 2.- Las normas del trabajo tienden a conseguir el equilibrio entre los factores de la producción y la justicia social, así como propiciar el trabajo digno o decente en todas las relaciones laborales. Se entiende por trabajo digno o decente aquél en el que se respeta plenamente la dignidad humana del trabajado